<a href="https://colab.research.google.com/github/mkunyosi/FLAI-CompeticaoML/blob/learning/Competi%C3%A7%C3%A3o_FLAI_T6_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6ª Competição FLAI

## A competição

*   Dado um conjunto de dados de funcionários de uma empresa, deseja-se prever quais são aqueles que irão sair da empresa nos próximos 2 anos.

### Dados

*   Escolaridade: grau de escolaridade do funcionário.
*   Entrada: ano de entrada na empresa. 
*   Cidade: cidade em que o funcionário mora.
*   Salário: classe de salário que o funcionário recebe (1 > 2 > *   3). 
*   Idade: idade em anos do funcionário. 
*   Sexo: sexo do funcionário. 
*   Projeto: se o funcionário já ficou sem projetos por 1 mês ou mais. 
*   Experiência: anos de experiência que o funcionário possui na área que atua.
*   Feedbacks: número de feedbacks negativos recebidos pelo funcionário.
*   Saiu: se o funcionário saiu da empresa antes de 2 anos de casa (0: não, 1: sim).

*   Conjunto de dados de treinamento: (2916, 10) contém a variável resposta “Saiu”. [Dados de treino](https://raw.githubusercontent.com/mkunyosi/FLAI-CompeticaoML/learning/treino.csv)
*   Conjunto de dados de teste: (1737, 9) não contém a variável resposta, a qual deve ser predita. [Dados de teste](https://raw.githubusercontent.com/mkunyosi/FLAI-CompeticaoML/learning/teste.csv)


### Métrica alvo

F1-SCORE - o modelo com o melhor índice leva a competição!

### Período para submissões

Início: 29/10/2021 </br>
Término: 28/11/2021 - 20:00

## Preparação do ambiente de trabalho

In [1]:
# executa este comando e depois reinicia o ambiente de execução no colab, para que o pycaret funcione. Adicionalemnte, inclui 
# o sweetviz para contribuir na Análise Exploratória de Dados.
!pip install pycaret
!pip install sweetviz

     |████████████████████████████████| 288 kB 23.5 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 15.5 MB 38.5 MB/s 
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 2.0 MB 40.2 MB/s 
     |████████████████████████████████| 113 kB 45.0 MB/s 
     |████████████████████████████████| 1.7 MB 36.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 261 kB 51.1 MB/s 
     |████████████████████████████████| 1.3 MB 34.7 MB/s 
     |████████████████████████████████| 6.8 MB 27.3 MB/s 
     |████████████████████████████████| 167 kB 63.3 MB/s 
     |████████████████████████████████| 675 kB 37.6 MB/s 
     |████████████████████████████████| 303 kB 47.0 MB/s 
     |████████████████████████████████| 3.1 MB 41.1 MB/s 
     |████████████████

In [2]:
import numpy as np
import pandas as pd

from random import randint


In [3]:
# Controle de debugging
varControl_RunExtraCode = False

## Importação dos dados

In [4]:
# importação dos dados no Github

treino = pd.read_csv('https://raw.githubusercontent.com/mkunyosi/FLAI-CompeticaoML/learning/treino.csv')
teste = pd.read_csv('https://raw.githubusercontent.com/mkunyosi/FLAI-CompeticaoML/learning/teste.csv')


In [5]:
# inspecão inicial dos dados
treino.head()

,Escolaridade,Entrada,Cidade,Salário,Idade,Sexo,Projeto,Experiência,Saiu,Feedbacks
0,Doutorado,2016,Natal,3,28,Feminino,Não,3,0,0
1,Doutorado,2014,Natal,3,36,Feminino,Não,3,0,1
2,Doutorado,2013,Natal,3,---,Masculino,Não,0,0,0
3,Doutorado,2016,Natal,3,---,Masculino,Não,2,1,0
4,Doutorado,2018,Natal,3,34,Masculino,Sim,5,1,0


## Pré-processamento dos dados

### Functions

In [6]:
# função recursiva determinar valores de campos não preenchidos
def reviewTarget(df, listVars, calcFunc='mean', level=0, inc=.1, verboseMode=False, sTrace=''):
    #Idade, Entrada, Salário, Sexo, Cidade, Experiência
    

    # set prefix for debugging
    txtPrint = "{:<" + str(level*4) + "}"
    txtPrint = txtPrint.format(" ")    
    
    # get target var
    listTarget = listVars[0]
    listVars.remove(listTarget)

    # start list of index for all row changed    
    allIndexChanged = []

    if verboseMode: print(txtPrint, "reviewTarget-begin", "level=", level, "vars=", listVars)        
    # set prefix for debugging
    txtPrint = "{:<" + str(level*4 + 1) + "}"
    txtPrint = txtPrint.format(" ")
    
    # check recursion base
    if df.shape[0] <= 1:
        if verboseMode: print(txtPrint, "recursion-base: few rows to eval", 'n_rows=', df.shape[0]) 
        if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars)  
        return -1, []
    # there are, at least, 2 rows => go ahead
    
    
    # check if there is least 1 row with null on target var and 1 row with values on target var
    if df[listTarget].isnull().sum() == 0 or df[listTarget].notnull().sum() == 0:
        if verboseMode: print(txtPrint, "recursion-base: no enough data to eval", 'n_nulls=', df[listTarget].isnull().sum(), 'n_values=',  df[listTarget].notnull().sum()) 
        if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars) 
        return -2, []
    # there are, at least, 2 rows (1 null and 1 with value) => go ahead


    if len(listVars) < 1:
        if verboseMode: print(txtPrint, "recursion-base: no var to filter data", 'listVars=', listVars) 
        if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars) 
        return 0, []

    # get first filter var from the list and sort it
    listItem = listVars[0]             
    listVars.remove(listItem)
    itemsSorted = np.sort(df[listItem].unique())

    localTrace = sTrace + '/' + listItem

    #check number of distinct items on the new var
    if itemsSorted.shape[0] < 1:
        #small list => don't need to eval each item
        #raise Exception("List of vars is too small. Review 'listVars' parameter.")
        if verboseMode: print(txtPrint, "recursion-base: no enough data to eval", 'listVars=', listVars) 
        if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars) 
        return -3, []
    
    #let´s eval each item on the filter var
    for i in itemsSorted:

        localTrace = localTrace + '(' + str(i) + ')'      
          
        df_filtered = df[df[listItem] == i].copy()         
        
        if verboseMode: print(txtPrint, "Working in new data_frame ", localTrace, ">", i)

        #call recursive function to eval next var
        if verboseMode: print(txtPrint, "recursion(begin)...", 'listVars=', listVars, 'item=', i)

        listVarsCpy = listVars.copy()                
        listVarsCpy.insert(0, listTarget)
        reviewTarget_ret, recItemsChanged = reviewTarget(df_filtered, listVarsCpy, calcFunc, level+1, inc + .1, verboseMode, localTrace)
        
        if verboseMode: print(txtPrint, "recursion(end): ", reviewTarget_ret, 'listVars=', listVars)

        if reviewTarget_ret >=0:
            if verboseMode: print(txtPrint, "update local data ", recItemsChanged)
            if len(recItemsChanged) > 0:
                if verboseMode: print(txtPrint, "")
                if verboseMode: print(txtPrint, "len(recItemsChanged)=", len(recItemsChanged))
                for j in recItemsChanged:
                    df.loc[j, listTarget] = df_filtered.loc[j, listTarget]
                    df.loc[j, 'trace'] = df_filtered.loc[j, 'trace']
                    allIndexChanged.append(j)
                    if verboseMode: print(txtPrint, "recItemsChanged=", j, ' newValue=', df.loc[j, listTarget])

            
        
            #set mean for other data
            if verboseMode: print(txtPrint, 'mean/mode calculus', 'type=', calcFunc, 'len(sample)=', len(df_filtered[listTarget]))
            #targetMean = round(np.mean(df_filtered[listTarget]),0) #+ inc
            if calcFunc == 'mean':
                targetMean = pd.to_numeric(round(np.mean(df_filtered[listTarget]),0), downcast='integer') #int(round(np.mean(df_filtered[listTarget]),0) )
            elif calcFunc == 'mode':
                targetMean = df_filtered[df_filtered['Projeto-new'].notna()]['Projeto-new'].mode()[0]
                #print('targetMean', targetMean)
                     
            else:
                if verboseMode: print(txtPrint, "invalid function", calcFunc) 
                return -4, []
            

            for j in df_filtered['Index']:
                if calcFunc == 'mean':
                    if pd.isna(df_filtered.loc[j, listTarget]):                                    
                        df.loc[j, listTarget] = pd.to_numeric(targetMean, downcast='integer') #int(targetMean )
                        df.loc[j, 'trace'] = ' idx=' + str(j) + 'mean=' + str(targetMean+ inc) + ' trace:' + sTrace
                        allIndexChanged.append(j)
                elif calcFunc == 'mode':
                    if pd.isna(df_filtered.loc[j, listTarget]):                
                        df.loc[j, listTarget] = targetMean 
                        df.loc[j, 'trace'] = ' idx=' + str(j) + 'mode=' + targetMean + str(inc) + ' trace:' + sTrace
                        allIndexChanged.append(j)



            if verboseMode: print(txtPrint, "targetMean=", targetMean, 'listTarget=',listTarget, ' index-upd=', allIndexChanged)


    if verboseMode: print(txtPrint, "reviewTarget-end", "level=", level, "vars=", listVars) 

    return 0, allIndexChanged

In [7]:
# target var: adjust values 
def evalTargetVar(df, targetVar, mode='hard', tiebreaker=0):

    targetAux = targetVar + '-new'

    for k in treino_new['Index']:
        if df.loc[k, 'action'] != "keep":
            df.loc[k, targetAux] = df.loc[k, 'Saiu']
        else:
            sum_0s = df.loc[k, '0s']
            sum_1s = df.loc[k, '1s']
            rowRef = df.loc[k, 'ref']
            if mode == 'hard':
                if sum_0s > sum_1s: newSet = 0
                elif sum_0s < sum_1s: newSet = 1
                elif tiebreaker == 0: newSet = 0
                elif tiebreaker == 1: newSet = 1
                else:
                    raise Exception("invalid tiebreaker:", tiebreaker)

                df.loc[k, targetAux] = newSet

                df[targetAux] = pd.to_numeric(df[targetAux], downcast='integer')
                                               
            elif mode == 'soft':
                if sum_0s == 0 and sum_1s == 0: newSet = 0.5
                else: newSet = sum_1s/(sum_0s + sum_1s)

                df.loc[k, targetAux] = newSet
            else:
                raise Exception("invalid mode:", mode)    



### Data processing

In [8]:
# Criação de features auxiliares para não alterar os dados originais
#
# Escolaridade-new: Cópia de Escolaridade e categorização de '---' para 'not PhD'
# Idade           : conversão de texto para número
# Idade-new       : Cópia de Idade
# Projeto-new     : Cópia de Projeto
# Index           : Índice para marcar as linhas (provavelmente tenha alguma forma mais limpa para ter acesso ao índice do elemento) 
treino["Escolaridade-new"] = treino["Escolaridade"].map(lambda x: x if x == "Doutorado" else "not PhD")
treino["Idade"] = treino["Idade"].map(lambda x: np.nan if x == "---" else pd.to_numeric(int(x), downcast='integer'))
treino["Idade-new"] = treino["Idade"].map(lambda x: x)
treino["Projeto-new"] = treino["Projeto"].map(lambda x: np.nan if x == "---" else x)
treino['Index'] = range(treino.shape[0])

teste["Escolaridade-new"] = teste["Escolaridade"].map(lambda x: x if x == "Doutorado" else "not PhD")
teste["Idade"] = teste["Idade"].map(lambda x: np.nan if x == "---" else pd.to_numeric(int(x), downcast='integer'))
teste["Idade-new"] = teste["Idade"].map(lambda x: x)
teste["Projeto-new"] = teste["Projeto"].map(lambda x: np.nan if x == "---" else x)
teste['Index'] = range(teste.shape[0])


In [9]:
#Acertar valores faltantes em 'Idade-new'
retCode, rowsChanged = reviewTarget(treino, ["Idade-new", "Entrada", "Salário", "Sexo", "Cidade", "Experiência"], calcFunc='mean', verboseMode=False)

retCode, rowsChanged = reviewTarget(teste, ["Idade-new", "Entrada", "Salário", "Sexo", "Cidade", "Experiência"], calcFunc='mean', verboseMode=False)


In [10]:
#Acertar valores faltantes em 'Projeto-new'
retCode, rowsChanged = reviewTarget(treino, ["Projeto-new", "Entrada", "Salário", "Sexo", "Cidade", "Experiência"], calcFunc='mode', verboseMode=False)

retCode, rowsChanged = reviewTarget(teste, ["Projeto-new", "Entrada", "Salário", "Sexo", "Cidade", "Experiência"], calcFunc='mode', verboseMode=False)

In [11]:
# identify similar rows and clean them

treino_new = pd.DataFrame()
teste_new = pd.DataFrame()

lstFrom = [ 'Index', 'Escolaridade-new', 'Entrada',	'Cidade', 'Salário','Idade-new', 'Sexo', 'Projeto-new', 'Experiência', 'Feedbacks', 'Saiu'] #			'Index'	'trace'	'Projeto-new']
lstVars = ['Index', 'Escolaridade', 'Entrada',	'Cidade', 'Salário','Idade', 'Sexo', 'Projeto', 'Experiência', 'Feedbacks', 'Saiu']

for i in range(len(lstVars)):
    treino_new[lstVars[i]] = treino[lstFrom[i]]
for i in range(len(lstVars)-1):
    teste_new[lstVars[i]] = teste[lstFrom[i]]

treino_dup = treino_new[treino_new.drop(['Index', 'Saiu'], axis=1).duplicated()].copy()

for i in treino_dup['Index']:
    rows_result = treino_new
    row = treino_dup[treino_dup['Index'] == i]
    for k in lstVars:
        if k not in ['Index', 'Saiu']:
          rows_result = rows_result[  rows_result[k] == treino_dup.loc[i][k]          ]
    
    sum_1s = rows_result['Saiu'].sum()
    sum_0s = rows_result['Saiu'].shape[0] - sum_1s
    rowRef = rows_result['Index'].min()
    for k in rows_result['Index']:
        treino_new.loc[k, '0s'] = sum_0s
        treino_new.loc[k, '1s'] = sum_1s
        treino_new.loc[k, 'ref'] = rowRef
        treino_new.loc[k, 'action'] = "keep" if rowRef == k else "del"        

#clean duplicate rows
print('shape-orig: ', treino_new.shape)
treino_new.drop(treino_new[treino_new['action'] == 'del'].index, inplace = True)
print('shape-clean: ', treino_new.shape)


shape-orig:  (2916, 15)
shape-clean:  (2102, 15)


In [12]:
evalTargetVar(treino_new, targetVar='Saiu',mode='hard', tiebreaker=0)
treino_new

,Index,Escolaridade,Entrada,Cidade,Salário,Idade,Sexo,Projeto,Experiência,Feedbacks,Saiu,0s,1s,ref,action,Saiu-new
0,0,Doutorado,2016,Natal,3,28.0,Feminino,Não,3,0,0,NaN,NaN,NaN,NaN,0.0
1,1,Doutorado,2014,Natal,3,36.0,Feminino,Não,3,1,0,NaN,NaN,NaN,NaN,0.0
2,2,Doutorado,2013,Natal,3,31.0,Masculino,Não,0,0,0,NaN,NaN,NaN,NaN,0.0
3,3,Doutorado,2016,Natal,3,30.0,Masculino,Não,2,0,1,1.0,1.0,3.0,keep,0.0
4,4,Doutorado,2018,Natal,3,34.0,Masculino,Sim,5,0,1,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908,2908,Doutorado,2014,João Pessoa,3,37.0,Masculino,Não,4,1,0,NaN,NaN,NaN,NaN,0.0
2909,2909,not PhD,2015,João Pessoa,2,29.0,Feminino,Não,2,1,1,NaN,NaN,NaN,NaN,1.0
2910,2910,not PhD,2014,Recife,3,28.0,Masculino,Não,2,0,0,NaN,NaN,NaN,NaN,0.0
2913,2913,not PhD,2016,Recife,3,31.0,Feminino,Sim,4,0,0,NaN,NaN,NaN,NaN,0.0


In [13]:
#checking changes
#C = np.where(cond, A, B)
treino_new['"Saiu" x "Saiu-new"'] = np.where(treino_new['Saiu'] == treino_new['Saiu-new'], True, False)
treino_new[treino_new['"Saiu" x "Saiu-new"'] == False].shape

(95, 17)

In [14]:
# acertar o valor da variável Saiu
treino_new['Saiu-old'] = treino_new['Saiu']
treino_new['Saiu'] = treino_new['Saiu-new']

## My Machine learning - Kunyosi's algorithm 🤓

In [15]:
# find neighbors for a entry
def findNeighbors(df, entry, idxEntry, lstFilters, varTarget,         verboseMode=False):
    # set prefix for debugging
    txtPrint = "{:<" + str(0) + "}"
    txtPrint = txtPrint.format(" ")

    if verboseMode: print(txtPrint, ">>>findNeighbors-begin>>>", "lstConstraints=", lstConstraints)      

    varFilters_cpy = lstFilters.copy()

    idx_neighbors = []
    value_targets = []

    localEntry = entry 

    for i in varFilters_cpy:
        localEntry_value = localEntry.loc[idxEntry, i]
        if verboseMode: print(txtPrint, 'filter=', i, 'localEntry.shape=', localEntry.shape)
        if verboseMode: print(txtPrint, 'value=', localEntry_value) #localEntry.head(1)[i][1])
        res = df[  df[i] == localEntry_value   ]
        
        if verboseMode: print(txtPrint, 'shape=', res.shape[0])
        if res.shape[0] > 1:
            df = res
    
    if df.shape[0] <= 0:
        #no fields for eval
        idx_neighbors = []
    else:
        #group neighbors
        if verboseMode: print(txtPrint, 'build list for neighbors')
        for i in df.index:
            idx_neighbors.append(i)
            if verboseMode: print(txtPrint, 'ref=', i)
            if verboseMode: print(txtPrint, 'target value=', df.loc[i][varTarget])
            value_targets.append(df.loc[i][varTarget])

    
    if verboseMode: print(txtPrint, "<<<findNeighbors-end<<<")    
    return idx_neighbors, value_targets    
    

In [16]:
# Teste da função findNeighbors
if varControl_RunExtraCode:
    lstConstraints = ['Entrada', 'Salário', 'Sexo', 'Cidade','Idade', 'Experiência']  #, 'Projeto', 'Feedbacks', 'Escolaridade']

    y_test = teste_new #[teste_new['Index'] == 1].copy()
    y_test_idx = 10

    nb_list, target_list = findNeighbors(treino_new, y_test, y_test_idx, lstConstraints, 'Saiu', verboseMode=True)
    print(nb_list)
    print(target_list)

In [17]:
#running my ML

def myPredict(X, y, varList, lstConstraints,         verboseMode=False):

    # set prefix for debugging
    #txtPrint = "{:<" + str(level*4) + "}"
    txtPrint = "{:<" + str(0) + "}"
    txtPrint = txtPrint.format(" ")

    if verboseMode: print(txtPrint, "myPredict-begin", "varList=", varList, "lstConstraints=", lstConstraints)      

    pred = []
    backlog = []

    XCopy = X.copy()

    varList_cpy = varList.copy()
    vTarget = varList_cpy[0]
    varList_cpy.remove(vTarget)

    ##for i in range(XCopy.shape[0]):
    XCopy['myIdx'] = XCopy.index

    yLen = y.shape[0] 

    
    for i in range(yLen):
        if i % 500 == 0: print('myPredict count=', i, '/', yLen)


        #test: pred.append(randint(0, 1))
        # step 1: search the same pattern on X
        df = XCopy

        if verboseMode: print(txtPrint, "")
        if verboseMode: print(txtPrint, 'y', y.loc[i])
        for j in varList_cpy:
            df = df[  df[j] == y.loc[i][j]          ]
            #print('shape-step:', j , df.shape)
            #if j not in ['Index', 'Saiu']:

        if verboseMode: print(txtPrint, 'shape-end' , df.shape)
        if df.shape[0] == 0:
            #not found
            #y_test = teste_new[teste_new['Index'] == 1].copy()
            #print(y.loc[i])
            #entry = y[y['Index'] == i].copy()
            #print(txtPrint, 'entry', entry) 
            nb_list, target_list = findNeighbors(treino_new, y, i, lstConstraints, vTarget, verboseMode)
            #print("***")
            sum_0s = 0
            sum_1s = 0
            for k in target_list:
              if k == 0: sum_0s += 1
              elif k == 1: sum_1s += 1
              else: sum_1s = sum_1s  
            
            voting = 0 if sum_0s >= sum_1s else 1
            pred.append(voting)
            backlog.append(nb_list)
        else:
            for k in range(df.shape[0]):
              if verboseMode: print(txtPrint, '   idx=', df.index[k])
              if verboseMode: print(txtPrint, '   dataframe')
              if verboseMode: print(txtPrint, XCopy.loc[df.index[k]]) #, df.loc[i, 'myIdx']])
            pred.append(XCopy.loc[df.index[k], vTarget])
            backlog.append(df.index[k])         
              


    if verboseMode: print(txtPrint, "myPredict-end", "pred=", pred)
    if verboseMode: print(txtPrint, "myPredict-end", "backlog=", backlog)


    return pred, backlog

## Running ML model

In [18]:
lstVars = ['Saiu', 'Escolaridade', 'Entrada',	'Cidade', 'Salário','Idade', 'Sexo', 'Projeto', 'Experiência', 'Feedbacks']
lstConstraints = ['Entrada', 'Salário', 'Sexo', 'Cidade','Idade', 'Experiência']  #, 'Projeto', 'Feedbacks', 'Escolaridade']

y_test = teste_new #.head(5).copy()
pred, bklog = myPredict(treino_new, y_test, lstVars, lstConstraints, verboseMode=False)


myPredict count= 0 / 1737
myPredict count= 500 / 1737
myPredict count= 1000 / 1737
myPredict count= 1500 / 1737


## Exporting results

In [19]:
subm_myPred = pd.DataFrame()

for i in range(len(pred)):
  subm_myPred.loc[i, 'Saiu'] = "{:.0f}".format(pred[i])
  #subm_myPred.loc[i, 'Log'] = str(bklog[i])

print(subm_myPred.shape)
subm_myPred.to_csv('subm_myPred.csv', index = False)

(1737, 1)


In [20]:
# Essa predição gerou um F1-Score = 0,80622
subm_myPred

,Saiu
0,0
1,0
2,0
3,0
4,1
...,...
1732,0
1733,1
1734,1
1735,0
